# Importing Required Libraries

In [1]:
import pandas as pd
import numpy as np
import xlrd
import os


## User driven Location of Source to target file

In [5]:
location = input(r"Enter the location of Source to target mapping (for e.g. : C:\Users\nsingh1\Documents\fun world\test_source_to_target.xlsx) \n :")

data = pd.read_excel(location)
print("\n"+"*"*10 + "File imported Successfully" + "*"*10)

print("\n Shape of the data\n ", data.shape)
print("\n Columns in Excels\n ", data.columns )


Enter the location of Source to target mapping (for e.g. : C:\Users\nsingh1\Documents\fun world\test_source_to_target.xlsx) \n :C:\Users\nsingh1\Documents\fun world\test_source_to_target.xlsx

**********File imported Successfully**********

 Shape of the data
  (19, 23)

 Columns in Excels
  Index(['Table_Name', 'Table_Defination', 'Table_Rational',
       'Table_Data_Analysis', 'Attribution_Level_Information', 'Column_Name',
       'Key', 'Data_Type', 'Size', 'Precision', 'Nullable', 'Default_Value',
       'Transformation', 'Need_review', 'Source_Column', 'Source_Table',
       'Other_Source_Tables', 'Column_Defination', 'Column_Rational',
       'Column_Is_SCD', 'Column_SCD_Reason', 'Column_Null_Percent',
       'Column_Group_Count'],
      dtype='object')


## Cleaning DataFrame for empty values

In [6]:
data.dropna(axis=0, subset=['Column_Name'], inplace=True)
print("Shape of the data after removal unwanted rows", data.shape)

# Replacing Pandas NA to String 'NA'

data = data.replace(np.nan, 'NA', regex=True)
print("Shape of the data after replacing NaN", data.shape)

Shape of the data after removal unwanted rows (19, 23)
Shape of the data after replacing NaN (19, 23)


## Final DDL Creation Script

In [7]:
file_name = data['Table_Name'][0].upper()+".sql"
print("DDL File Name : ",file_name)
print("DDL File Path : {}\{}\n".format(os.getcwd(),
                                   file_name))
with open(file_name, 'w') as f:
    
    #Table related Analysis
    f.write("/* \n\n Defination: {}\n\n Rational: {} \n\n Data Analysis: \n\n {}\n\n Attribution Level Information: \n\n{}\n\n\n */ \n\n\n  Create Table Wars.bi.{}  \n\n   ( \n".format(data['Table_Defination'][0],
                                                                                                                            data['Table_Rational'][0],
                                                                                                                            data['Table_Data_Analysis'][0],
                                                                                                                            data['Attribution_Level_Information'][0],
                                                                                                                            data['Table_Name'][0]))
    column_count = 0
    #Column related Analysis
    for elem in range(len(data)):

        column_name =data['Column_Name'][elem].upper()
        data_type = data['Data_Type'][elem].upper()

        # Size and Precision Block 
        if data['Size'][elem] == 'NA' :   # Checking whether the size of the column is present or not for e.g. integer has no size and precision whereas decimal has 
            precisions=""

        else:
            size = str(int(data['Size'][elem]))
            
    #       For precision 
            if data['Precision'][elem] == 'NA':
                precisions = "( "+ size + " )"

            else:            
                precisions = "( " + size + "," + str(int(data['Precision'][elem])) + " )"


        # Nullability Check, whether the column is null or not
        if data['Nullable'][elem].lower() == "no":
            Null_check = " NOT NULL ,"

        else:
            Null_check = " ,"
      
        #Comment String with Column Analysis
        comment = "  --" + data['Column_Defination'][elem] + " | " + data['Column_Defination'][elem] + " | "  +  data['Column_Rational'][elem] + " | "  + data['Column_Is_SCD'][elem] + " | " + data['Column_SCD_Reason'][elem] + " | "  +    str(data['Column_Null_Percent'][elem]) + "% | " + str(data['Column_Group_Count'][elem])
        
        # Column row in DDL
        text = "{:>45}{:>18}{:>12}{:>12}{:>100}".format(column_name,
                                                        data_type,
                                                        precisions,
                                                        Null_check,
                                                        comment)
        f.write(text)
        f.write("\n")
        column_count = column_count + 1
    f.write("\n\n")
    
    # Constraint Block in DDl
    constraint_cnt = 0 
    for index in range(len(data)):
        
        if data['Key'][index].upper()=='PK':
            line ="  CONSTRAINT PK_{} PRIMARY KEY NONCLUSTERED ({})".format(data['Table_Name'][0],
                                                                          data['Column_Name'][index].upper())
            f.write(line+"\n")
            constraint_cnt = constraint_cnt +1
            
        elif data['Key'][index].upper()=='FK':
            f.write(" ,")
            line = "CONSTRAINT FK_{}_{} FOREIGN KEY ({}) REFERENCES {} ({})".format(data['Table_Name'][0],
                                                                                     data['Column_Name'][index].upper(),
                                                                                     data['Column_Name'][index].upper(),
                                                                                     data['Source_Table'][index],
                                                                                     data['Source_Column'][index].upper())
            f.write(line+"\n")
            constraint_cnt = constraint_cnt +1
            
        else:
            continue
            
    f.write("\n);")
    f.close()

print("\n Total Column inserted : ", column_count)
print("\n Total Constraints (Inc. 1 Primary Key constraint) : ", constraint_cnt)
print("\n DDL Created Successfully")

    
    

DDL File Name :  F_PAYMENT_TRANSACTION.sql
DDL File Path : C:\Users\nsingh1\Documents\fun world\F_PAYMENT_TRANSACTION.sql


 Total Column inserted :  19

 Total Constraints (Inc. 1 Primary Key constraint) :  8

 DDL Created Successfully
